In [25]:
#!/usr/bin/env python3
"""
Date : 2019, APRIL
Author : Apiwit Theeraporn
"""

'\nDate : 2019, APRIL\nAuthor : Apiwit Theeraporn\n'

In [26]:
# import libraries

In [27]:
import os
from io import BytesIO

In [28]:
# keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD,RMSprop,adam
from keras.utils import np_utils
from keras.preprocessing import image
from keras.callbacks import EarlyStopping, ModelCheckpoint

# use tensorflow backend
import tensorflow

In [29]:
import numpy as np

import PIL

import cv2

import IPython
from IPython.display import display,Image,clear_output

In [30]:
# pre-deterministic input

In [31]:
train_dir = 'processed_images/'

# input image dimensions
img_rows, img_cols = 215, 240

# Batch_size to train
batch_size = 8

# Number of epochs to train
nb_epoch = 20  

# Total number of convolutional filters to use ~ batch
nb_filters = 8

# Max pooling
nb_pool = 2

# Size of convolution kernel
nb_conv = 5

kernel_size = (nb_conv,nb_conv);

In [32]:
# pre-deterministic function

In [33]:
# Display : This method instaed of cv2.imageShow
def iPythonDisplay(img_arr, fmt='jpeg'):
    with BytesIO() as output:
        with PIL.Image.fromarray(img_arr) as img:
            img.save(output,format=fmt);
        img_byte = output.getvalue()
    display(Image(data=img_byte))

In [34]:
# Keras load images from folder

In [35]:
# flow_from_directory will first load data from it's config
"""
The function will run after the image is resized and augmented. 
The function should take one argument: an image (Numpy tensor with rank 3), 
and should output a Numpy tensor with the same shape.
"""
datagen = image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2 # reserve some data for validation
)

# number of channels
# For 'grayscale' use 1 ,and for 'rgb' images use 3
img_channels = 1

# import first then transform
processed_train_generator = datagen.flow_from_directory(
    directory=train_dir,
    batch_size=batch_size,
    target_size = (img_rows, img_cols),
    class_mode="categorical",
    color_mode = 'grayscale',
    subset='training',
)

processed_valid_generator = datagen.flow_from_directory(
    directory=train_dir,
    batch_size=batch_size,
    target_size = (img_rows, img_cols),
    class_mode="categorical",
    color_mode = 'grayscale',
    subset='validation',
)


nb_classes = len(processed_train_generator.class_indices)
print('all classes : ',processed_valid_generator.class_indices)
print('got total class =',len(processed_train_generator.class_indices));

Found 32 images belonging to 4 classes.
Found 8 images belonging to 4 classes.
all classes :  {'buff': 0, 'fist': 1, 'luv': 2, 'rock': 3}
got total class = 4


In [36]:
# define model

In [37]:
model = Sequential()

model.add(Conv2D(
    nb_filters, 
    kernel_size,
    input_shape=(img_rows, img_cols,img_channels)))
    # input_shape=(128, 128, 3) for 128x128 RGB pictures in data_format="channels_last"
convout1 = Activation('relu')
model.add(convout1)
model.add(Conv2D(nb_filters, (nb_conv, nb_conv)))
convout2 = Activation('relu')
model.add(convout2)
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 211, 236, 8)       208       
_________________________________________________________________
activation_13 (Activation)   (None, 211, 236, 8)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 207, 232, 8)       1608      
_________________________________________________________________
activation_14 (Activation)   (None, 207, 232, 8)       0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 103, 116, 8)       0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 103, 116, 8)       0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 95584)            

In [41]:
STEP_SIZE_TRAIN=processed_train_generator.batch_size

STAMP = 'best_gesture_iteration'
early_stopping =EarlyStopping(monitor='val_loss', patience=5)
bst_model_path = STAMP + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=False)


model.fit_generator(
        processed_train_generator,
        epochs=nb_epoch,
        steps_per_epoch=STEP_SIZE_TRAIN,
        verbose=1,
        validation_data=processed_valid_generator,
        use_multiprocessing = True,
        workers = 2, #CPU 4 cores,
        callbacks= [early_stopping, model_checkpoint]
        )

Epoch 1/20
8/8 [==============================] - 7s 877ms/step - loss: 1.3663 - acc: 0.4531 - val_loss: 1.0714 - val_acc: 0.7500
Epoch 2/20
8/8 [==============================] - 6s 722ms/step - loss: 0.6292 - acc: 0.8594 - val_loss: 0.4944 - val_acc: 0.7500
Epoch 3/20
8/8 [==============================] - 6s 726ms/step - loss: 0.1039 - acc: 0.9844 - val_loss: 0.2631 - val_acc: 0.8750
Epoch 4/20
8/8 [==============================] - 6s 732ms/step - loss: 0.0277 - acc: 1.0000 - val_loss: 0.2556 - val_acc: 0.8750
Epoch 5/20
8/8 [==============================] - 7s 834ms/step - loss: 0.0125 - acc: 1.0000 - val_loss: 0.3936 - val_acc: 0.8750
Epoch 6/20
8/8 [==============================] - 6s 758ms/step - loss: 0.0034 - acc: 1.0000 - val_loss: 0.3226 - val_acc: 0.8750
Epoch 7/20
8/8 [==============================] - 6s 732ms/step - loss: 0.0024 - acc: 1.0000 - val_loss: 0.3687 - val_acc: 0.8750
Epoch 8/20
8/8 [==============================] - 6s 735ms/step - loss: 0.0017 - acc: 1.00

In [42]:
# predict data ex. in sample

In [43]:
#load model
# from keras.models import load_model
# model = load_model('best_gesture_iteration.h5')

In [46]:
predict = model.predict_generator(
    processed_valid_generator, 
    steps=None, 
    callbacks=None, 
    workers=2, 
    use_multiprocessing=True, 
    verbose=1)

print('all classes : ',processed_valid_generator.class_indices)
np.round(predict)

1/1 [==============================] - 0s 311ms/step
all classes :  {'buff': 0, 'fist': 1, 'luv': 2, 'rock': 3}


array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)